# Задания

In [2]:
import numpy as np
import pandas as pd

## Задание 1

Загрузи данные о посещениях заведений. В них содержится информация о регистрации пользователя в заведениях 
и геолокация этих заведений. Очисти данные от записей с пропусками. \
Выведи количество записей после очистки. 

In [3]:
# Загрузка данных из файла .dat с указанием типа данных для каждого столбца
data = pd.read_csv("../datasets/checkins.dat", delimiter='|', skipinitialspace=True, quotechar="'", na_values='', 
                   dtype={'id': int, 'user_id': int, 'venue_id': int, 'latitude': float, 'longitude': float, 'created_at': 'datetime64'})
data.columns = data.columns.str.strip()


# Очистка данных от записей с пропусками
data_cleaned = data.dropna(subset=['latitude'])

# Вывод количества записей после очистки
print(f"Количество записей после очистки: {len(data_cleaned)}, а было {len(data)}")

Количество записей после очистки: 396634, а было 1021968


C:\Users\sv_k7\AppData\Local\Temp\ipykernel_17980\2040007698.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../datasets/checkins.dat", delimiter='|', skipinitialspace=True, quotechar="'", na_values='',


## Задание 2

Эти данные содержат информацию о заведениях со всего мира. С помощью геолокаций и библиотеки [Reverse Geocoder](https://github.com/thampiman/reverse-geocoder),
узнай страну для каждой геопозиции. \
Узнай **название** второй страны по количеству записей.

In [5]:
import reverse_geocoder as rg
import pycountry

# Кортежи с координатами
coordinates = list(zip(data_cleaned['latitude'], data_cleaned['longitude']))

# Поиск по координатам
geo_data = rg.search(coordinates)

# Заполнение кодами стран
data_cleaned.loc[:, 'country'] = [country['cc'] for country in geo_data]

# Код страны, второй по количеству записей
country_counts = data_cleaned['country'].value_counts()
second_country_code = country_counts.index[1]

# Получение названия второй страны по количеству записей
second_country = pycountry.countries.get(alpha_2=second_country_code).name

print(f"Название второй страны по количеству записей: {second_country}")


Название второй страны по количеству записей: Indonesia


## Задание 3

Нас будут интересовать только американские локации. Очисти данные от локаций, находящихся в других странах. 
Также, чтобы сократить количество, геолокаций оставь в выборке только 50 самых часто встречаемых заведений (venue). \
Выведи количество локаций, оставшихся после этих очисток.

In [6]:
# Очистка данных от локаций, не относящихся к США
data_us = data_cleaned[data_cleaned['country'] == 'US']

# Получение 50 самых часто встречаемых заведений
top_venues = data_us['venue_id'].value_counts().nlargest(50)

# Оставление только выбранных заведений
data_filtered = data_us[data_us['venue_id'].isin(top_venues.index)]

# Вывод количества оставшихся локаций
num_locations = len(data_filtered)
print(f"Количество оставшихся локаций: {num_locations}")

top_venues = data_cleaned['venue_id'].value_counts().head(5)

print("ID локаций и их количество упоминаний в порядке убывания:")
for venue_id, count in top_venues.items():
    print(f"ID: {venue_id}, Количество упоминаний: {count}")


Количество оставшихся локаций: 162099
ID локаций и их количество упоминаний в порядке убывания:
ID: 5222.0, Количество упоминаний: 12739
ID: 7620.0, Количество упоминаний: 9716
ID: 2297.0, Количество упоминаний: 9321
ID: 11195.0, Количество упоминаний: 8286
ID: 60.0, Количество упоминаний: 7195


## Задание 4

Перейдем к задаче кластеризации. Воспользуйся алгоритмом [Mean Shift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)
для кластеризации локаций. Параметрами укажи `MeanShift(bandwidth=0.1, bin_seeding=True)`. 

    `bandwidth=0.1` - это ширина ядра кластеризации. Для средних широт США - это порядка 5-10 км. 
    `bin_seeding=True` - для ускорения работы алгоритма.
    
Выведи количество кластеров, которые у тебя получились в результате кластеризации.

In [7]:
from sklearn.cluster import MeanShift

# Кластеризация локаций с помощью Mean Shift
bandwidth = 0.1
bin_seeding = True
meanshift = MeanShift(bandwidth=bandwidth, bin_seeding=bin_seeding)
clusters = meanshift.fit_predict(data_filtered[['latitude', 'longitude']])

# Количество кластеров
num_clusters = len(set(clusters))
print(f"Количество кластеров: {num_clusters}")

Количество кластеров: 2846


## Задание 5

Центры полученных кластеров - это потенциальные места установки банеров компании. Теперь хотелось бы найти те центры кластеров, 
которые наиболее близко находятся к офисам продаж компании. \
Загрузи [данные по координатам офисов компании](datasets/offices.csv). Для каждого офиса найди 5 самых ближайших к нему центров кластеров. 
(Пренебрежем тем, что Земля круглая и рассчитаем Евклидово расстояние).
У компании 11 офисов, значит у нас должно получится 55 мест установки баннеров. \
Выведи координаты установки баннера, который ближе всего находится к офису компании.

In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import MeanShift

# Загрузка данных об офисах
offices_data = pd.read_csv('../datasets/offices.csv')

# Создание новых столбцов для хранения координат и расстояния ближайшего кластера
offices_data['closest_cluster_coordinates'] = None
offices_data['distance_to_closest_cluster'] = None
offices_data['closest_cluster_indices'] = None
offices_data['banner_latitude'] = None  # Новый столбец для хранения широты баннеров
offices_data['banner_longitude'] = None  # Новый столбец для хранения долготы баннеров

# Перебор каждого офиса
for index, row in offices_data.iterrows():
    office_latitude = row['latitude']
    office_longitude = row['longitude']
    
    # Вычисление расстояний от офиса до каждого центра кластера
    distances = []
    for cluster_center in meanshift.cluster_centers_:
        cluster_latitude = cluster_center[0]
        cluster_longitude = cluster_center[1]
        
        # Вычисление евклидова расстояния между офисом и центром кластера
        distance = ((office_latitude - cluster_latitude)**2 + (office_longitude - cluster_longitude)**2)**0.5
        distances.append(distance)
    
    # Находим индексы 5 ближайших кластеров
    closest_cluster_indices = np.argsort(distances)[:5]
    
    # Находим индекс самого близкого кластера
    closest_cluster_index = np.argmin(distances)
    
    # Сохранение координат и расстояния самого близкого кластера в соответствующих столбцах
    closest_cluster_coordinates = meanshift.cluster_centers_[closest_cluster_index]
    closest_cluster_distance = distances[closest_cluster_index]
    banner_coordinates = meanshift.cluster_centers_[closest_cluster_indices]  # Координаты баннеров
    banner_latitudes = banner_coordinates[:, 0]  # Широты баннеров
    banner_longitudes = banner_coordinates[:, 1]  # Долготы баннеров
    offices_data.at[index, 'closest_cluster_coordinates'] = closest_cluster_coordinates
    offices_data.at[index, 'distance_to_closest_cluster'] = closest_cluster_distance
    offices_data.at[index, 'closest_cluster_indices'] = closest_cluster_indices
    offices_data.at[index, 'banner_latitude'] = banner_latitudes  # Сохранение широты баннеров
    offices_data.at[index, 'banner_longitude'] = banner_longitudes  # Сохранение долготы баннеров

    # Вывод результатов (координаты и дистанция самого близкого кластера)
dis = 1
office = 0
coordy = 0
for index, row in offices_data.iterrows():
    if dis > row['distance_to_closest_cluster']:
        office = index + 1
        dis = row['distance_to_closest_cluster']
        coordy = row['closest_cluster_coordinates']

print(f"Координаты и дистанция самого близкого кластера к офису {office}: {coordy}, {dis}")    

Координаты и дистанция самого близкого кластера к офису 10: [ 32.78531778 -79.92474241], 0.004092113620846756


## Задание 6

С помощью функции [scatter_mapbox](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html) 
отметь точки установки баннеров. У тебя должна получится такая картинка.
Цветом точки укажи к какому офису будет относиться этот баннер. 
>Цвет легенды может отличаться от референса

<center><img src="../misc/images/task_6.png" width="800" height="800"/> <center/>

In [23]:
offices_data = offices_data.rename(columns={'Unnamed: 0': 'office_name'})
fig = go.Figure()

# Добавление точек офисов
fig.add_trace(
    go.Scattermapbox(
        lat=offices_data['latitude'],
        lon=offices_data['longitude'],
        mode='markers',
        marker=dict(size=10, color='blue'),
        name='Offices'
    )
)

# Добавление точек баннеров
for _, row in offices_data.iterrows():
    banner_latitudes = row['banner_latitude']
    banner_longitudes = row['banner_longitude']
    
    fig.add_trace(
        go.Scattermapbox(
            lat=banner_latitudes,
            lon=banner_longitudes,
            mode='markers',
            marker=dict(size=5, color='red'),
            name='Banners'
        )
    )

# Настройка внешнего вида карты
fig.update_layout(
    mapbox=dict(
        style='carto-positron',
        center=dict(lat=44.5, lon=280.4),
        zoom=2
    ),
    title='Расположение баннеров и офисов',
    showlegend=True
)

# Отображение карты
fig.show()